In [1]:
import numpy as np
import heapq

In [2]:
#解码数据集 函数为CIFAR-10提供的python3版本
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#创建训练集和测试集
def CreatData(path):
    #依次加载CIFAR-10的5个batch_data,并将其合并为traindata和traindlabels
    x=[]
    y=[]
    for i in range(1,6):
        batch_path=path + 'data_batch_%d'%(i) #每个batch的地址
        batch_dict=unpickle(batch_path) #解码每个batch
        train_batch=batch_dict[b'data'].astype('float') #将每个batch的data部分以float形式存储于train_batch变量
        train_labels=np.array(batch_dict[b'labels']) #将每个batch的label部分以np.array的形式存储于train_labels变量
        x.append(train_batch)
        y.append(train_labels)
    #将5个训练样本batch(10000,3072)合并为(50000,3072)，标签合并为(50000,1)
    #np.concatenate默认axis=0:按行合并，axis=1则为:按列合并
    traindata=np.concatenate(x)
    trainlabels=np.concatenate(y)
    
    #加载测试集
    testpath=path + 'test_batch' #test_batch的地址
    test_dict=unpickle(testpath) #解码test_batch
    testdata=test_dict[b'data'].astype('float') #将test_dict的data部分以float形式存储于testdata变量
    testlabels=np.array(test_dict[b'labels']) #将test_dict的labels部分以np.array形式存储于testlabels变量
    
    #将训练集数据、训练集标签、测试集数据、测试集标签返回
    return traindata,trainlabels,testdata,testlabels

In [3]:
class neuralNetwork:
    def __init__(self):
        self.w1 = (np.random.rand(100,3072) - 0.5) / np.sqrt(100 * 3072)#第一层权重，维度变化为(100,3072)*(3072,1)=(100,1)
        self.w2 = (np.random.rand(20,100) - 0.5) / np.sqrt(20 * 100)#第二层权重，维度变化为(20,100)*(100,1)=(20,1)
        self.w3 = (np.random.rand(10,20) - 0.5) / np.sqrt(10 * 20)#第三层权重，维度变化为(10,20)*(20,1)=(10,1)
        self.b1 = np.random.rand(100,1) - 0.5#第一层偏执，维度为(100,1)
        self.b2 = np.random.rand(20,1) - 0.5#第二层偏执，维度为(20,1)
        self.b3 = np.random.rand(10,1) - 0.5#第三层偏执，维度为(10,1)
        self.lr = 0.01#设置学习率
        pass
    
    def predataprocess(self,data):#数据预处理
        data -= self.mean
        data /= np.std(data)
        return data
    
    def sigmoid(self,x):#定义sigmoid激活函数
        return 1.0 / (1.0 + np.exp(-x))
    
    def softmax(self,out):#定义softmax函数
        c = np.max(out)
        temp = np.exp(out - c)
        return temp / np.sum(temp)
    
    def forward(self,x,w1,w2,w3,b1,b2,b3):#定义前向传播
        x = x.reshape(len(x),1)
        h1 = np.dot(w1,x) + b1
        sig_h1 = self.sigmoid(h1)
        h2 = np.dot(w2,sig_h1) + b2
        sig_h2 = self.sigmoid(h2)
        h3 = np.dot(w3,sig_h2) + b3
        out = self.softmax(h3)
        return out
    
    def CrossEntropyLoss(self,x,w1,w2,w3,b1,b2,b3,y):#定义交叉熵
        out = self.forward(x,w1,w2,w3,b1,b2,b3)
        loss = -np.log(out[y,0])
        return loss,out
    
    def gradient(self,x,w1,w2,w3,b1,b2,b3,y):#用笨办法求梯度(每个权重和偏执做微小改变，求梯度)
        loss,out = self.CrossEntropyLoss(x,w1,w2,w3,b1,b2,b3,y)
        dw1 = np.zeros(w1.shape)
        dw2 = np.zeros(w2.shape)
        dw3 = np.zeros(w3.shape)
        db1 = np.zeros(b1.shape)
        db2 = np.zeros(b2.shape)
        db3 = np.zeros(b3.shape)
        
        tempw1 = w1.copy()
        tempw2 = w2.copy()
        tempw3 = w3.copy()
        tempb1 = b1.copy()
        tempb2 = b2.copy()
        tempb3 = b3.copy()
        
        h = 0.0001
        for i in range(w1.shape[0]):
            for j in range(w1.shape[1]):
                tempw1[i][j] += h
                temploss,out = self.CrossEntropyLoss(x,tempw1,tempw2,tempw3,tempb1,tempb2,tempb3,y)
                dw1[i][j] = (temploss - loss) / h
                tempw1[i][j] -= h
        for i in range(w2.shape[0]):
            for j in range(w2.shape[1]):
                tempw2[i][j] += h
                temploss,out = self.CrossEntropyLoss(x,tempw1,tempw2,tempw3,tempb1,tempb2,tempb3,y)
                dw2[i][j] = (temploss - loss) / h
                tempw2[i][j] -= h
        for i in range(w3.shape[0]):
            for j in range(w3.shape[1]):
                tempw3[i][j] += h
                temploss,out = self.CrossEntropyLoss(x,tempw1,tempw2,tempw3,tempb1,tempb2,tempb3,y)
                dw3[i][j] = (temploss - loss) / h
                tempw3[i][j] -= h
                
        for i in range(b1.shape[0]):
            tempb1[i][0] += h
            temploss,out = self.CrossEntropyLoss(x,tempw1,tempw2,tempw3,tempb1,tempb2,tempb3,y)
            db1[i][0] = (temploss - loss) / h
            tempb1[i][0] -= h
        for i in range(b2.shape[0]):
            tempb2[i][0] += h
            temploss,out = self.CrossEntropyLoss(x,tempw1,tempw2,tempw3,tempb1,tempb2,tempb3,y)
            db2[i][0] = (temploss - loss) / h
            tempb2[i][0] -= h
        for i in range(b3.shape[0]):
            tempb3[i][0] += h
            temploss,out = self.CrossEntropyLoss(x,tempw1,tempw2,tempw3,tempb1,tempb2,tempb3,y)
            db3[i][0] = (temploss - loss) / h
            tempb3[i][0] -= h
        
        return loss,dw1,dw2,dw3,db1,db2,db3
        
    def loaddata(self,traindata,trainlabels,testdata,testlabels):#读入训练数据和测试数据，并进行数据预处理(注意:测试集均值用的训练集的平均值)
        self.mean = np.mean(traindata)
        self.imgnum = traindata.shape[0]
        
        data = traindata.copy()
        data = self.predataprocess(data)
        self.traindata = data
        self.trainlabels = trainlabels
        
        data = testdata.copy()
        data = self.predataprocess(data)
        self.testdata = data
        self.testlabels = testlabels
    
    def train(self):#训练，求梯度，更新权重，输出损失
        Loss = np.zeros(self.imgnum)
        for i in range(self.imgnum):
            x = self.traindata[i,:]
            y = self.trainlabels[i]
            Loss[i],dw1,dw2,dw3,db1,db2,db3 = self.gradient(x,self.w1,self.w2,self.w3,self.b1,self.b2,self.b3,y)
            self.w1 -= self.lr * dw1
            self.w2 -= self.lr * dw2
            self.w3 -= self.lr * dw3
            self.b1 -= self.lr * db1
            self.b2 -= self.lr * db2
            self.b3 -= self.lr * db3
            print(i,Loss[i])
        return Loss
 
    def predict(self):
        #得到测试集总图片数，并保存到testimg_num内
        testimg_num = self.testdata.shape[0]
        #创建一个维度为(testimg_num，)的np.array，用于存储预测的标签
        predlabels = np.zeros(testimg_num, dtype = self.trainlabels.dtype)
        Loss = 0
        #遍历训练集
        for i in range(testimg_num):
            x = self.testdata[i,:]
            y = self.testlabels[i]
            
            Li,out = self.CrossEntropyLoss(x,self.w1,self.w2,self.w3,self.b1,self.b2,self.b3,y)
            predlabels[i] = np.argmax(out)#取最近k张图片的下标
            #heapq.nsmallest(k, distances)返回distances最小的k个元素
            #map(list(distances).index, heapq.nsmallest(k, distances))则返回distances最小的k个元素对应的索引，调用list.index()函数来寻找
            #mindistances_index最终以list形式储存最近的k张图片的索引
            Loss += Li
        Loss / testimg_num
        return Loss,predlabels

In [4]:
#读取训练集和测试集的数据和标签
traindata,trainlabels,testdata,testlabels = CreatData("E:/dataset/cifar-10-batches-py/")

nn = neuralNetwork()#实例化网络
nn.loaddata(traindata,trainlabels,testdata,testlabels)#读入数据
trainloss = nn.train()#训练
testloss,predlabels = nn.predict()#预测

0 2.551283542090421
1 1.7960153261880238
2 1.7522959076326408
3 2.6273146614870138
4 2.664140916393232
5 2.609335568620913
6 2.4489677232390186
7 2.6916819751733048
8 2.3978152511573847
9 2.0608032725242404
10 2.5908424327091906
11 2.6471723498473376
12 2.59272293073303
13 2.418774678841763
14 1.8005080456875815
15 1.756838051329067
16 1.7139378098218558
17 2.068914699251568
18 2.3994764601718455
19 2.5796635091456777
20 2.580640724429647
21 2.0380219057721134
22 2.534026810594865
23 2.4804607222896036
24 2.3683780255156446
25 2.4316078257799587
26 2.0158747961744656
27 2.6271593034634604
28 2.55353155043485
29 2.155547118919985
30 2.1060874784496075
31 1.8032147224518131
32 2.6598771137314206
33 2.013869646584096
34 2.5269723912325617
35 2.087059343645789
36 1.9817307735544343
37 2.645762327487263
38 1.9409748518760763
39 1.8948157404416308
40 2.6292188900723503
41 2.4044672245522696
42 2.3522562396105076
43 2.6130115457129666
44 2.6497829498401217
45 2.595395374247553
46 2.5414402810

In [6]:
print('accuracy:',np.mean(predlabels == testlabels)) #计算测试集准确率

accuracy: 0.3202
